In [5]:
from pgljupyter import *

Run a module

In [6]:
%%lpy -w 10 -a True
from random import random 
MAX_AGE, dr = 10, 0.03 # constants
module Apex(age), Internode(length,radius)
Axiom: @Gc Apex(0)
derivation length: 5
production:
Apex(age) :
  if age < MAX_AGE:
    produce Internode(1+ random(),0.03) /(137)[+(30)Apex(age+1)]Apex(age+1)

Internode(l,r) --> Internode(l,r+dr)
interpretation:
Internode(l,r) --> _(r) F(l)
endlsystem

LsystemWidget(animate=True, derivationLength=6, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe5R…